In [1]:
import numpy as np
import requests
import json
import csv
import time
import datetime
import os
import pandas as pd
import pprint

In [2]:
def getApiData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title=' + str(query) + '&size=1000&after=' + str(
        after) + '&before=' + str(before) + '&subreddit=' + str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [3]:
def collectData(subm):
    sub_data = list()  # list to store data points
    title = subm['title']
    url = subm['url']
    try:
        # if flair is available then get it, else set 'NaN'
        flair = subm['link_flair_text']
    except KeyError:
        flair = 'NaN'
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    try:
        # if selftext is available then get it, else set it empty
        selftext = subm['selftext']
        list_of_empty_markers = ['[removed]', '[deleted]']
        # many times selftext would be removed or deleted, if thats the case then set it empty
        if selftext in list_of_empty_markers:
            selftext = ''
    except:
        selftext = ''
    created = datetime.datetime.fromtimestamp(subm['created_utc'])  # 1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    sub_data.append((sub_id, title, selftext, url, author, score, created, numComms, permalink, flair))
    sub_stats[sub_id] = sub_data
    # print(sub_stats)


In [4]:
def write_subs_to_file(filename):
    upload_count = 0
    if os.path.exists(filename):
        keep_header = False
    else:
        keep_header = True

    with open(filename, 'a', newline='', encoding='utf-8') as file:
        a = csv.writer(file, delimiter=',')
        headers = ['post_id', 'title', 'selftext', 'url', 'author', 'score', 'publish_date', 'num_of_comments',
                   'permalink', 'flair']
        if keep_header:
            a.writerow(headers)
        for sub in sub_stats:
            a.writerow(sub_stats[sub][0])
            upload_count += 1
        # print(str(upload_count) + ' submissions have been uploaded')
        


In [8]:

# Download reddit posts from sub_reddit with keywords given by key_word

sub_reddit = 'bitcoin'
key_word = 'bitcoin'

output_filename = 'reddit_data.csv'
# search all the posts from start_date to end_date overall
start_date = datetime.datetime(2020, 5, 12, 0)
end_date = datetime.datetime(2020, 7, 31, 0)

# in each itration get reddit posts for one day, to avoid getting blocked by server
one_day = datetime.timedelta(hours=24)
after_date = start_date
after = str(int(after_date.timestamp()))
before_date = start_date + one_day
before = str(int(before_date.timestamp()))

while after_date < end_date:
    print('-' * 80)
    print(after_date, ' -> ', before_date)
    print('-' * 80)

    sub_count = 0
    sub_stats = {}

    data = getApiData(key_word, after, before, sub_reddit)

    max_count = 100
    count = 0
    while len(data) > 0 and count < max_count:
        print('count ', count)
        for submission in data:
            collectData(submission)
            sub_count += 1

        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = getApiData(key_word, after, before, sub_reddit)
        # print(data)
        # print(data['data'][0]['author'])
        count = count + 1

    # keep saving data collected in each iteration
    write_subs_to_file(output_filename)

    # move to next day
    after_date += one_day
    after = str(int(after_date.timestamp()))
    before_date += one_day
    before = str(int(before_date.timestamp()))

    # randomly sleep before starting next iteration
    time.sleep(np.random.randint(1, 3))


--------------------------------------------------------------------------------
2020-05-12 00:00:00  ->  2020-05-13 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1589221800&before=1589308200&subreddit=bitcoin
count  0
100
2020-05-12 06:56:54
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1589246814&before=1589308200&subreddit=bitcoin
count  1
100
2020-05-12 23:20:36
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1589305836&before=1589308200&subreddit=bitcoin
count  2
1
2020-05-12 23:54:42
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1589307882&before=1589308200&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-05-13 00:00:00  ->  2020-05-14 00:00:00
----------------------------------------------------------

count  0
96
2020-05-25 23:48:54
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1590430734&before=1590431400&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-05-26 00:00:00  ->  2020-05-27 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1590431400&before=1590517800&subreddit=bitcoin
count  0
100
2020-05-26 22:45:48
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1590513348&before=1590517800&subreddit=bitcoin
count  1
8
2020-05-26 23:26:43
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1590515803&before=1590517800&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-05-27 00:00:00  ->  2020-05-28 00:00:00
-----------------------------------------------------------

count  0
97
2020-06-09 23:46:13
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1591726573&before=1591727400&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-06-10 00:00:00  ->  2020-06-11 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1591727400&before=1591813800&subreddit=bitcoin
count  0
92
2020-06-10 23:50:57
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1591813257&before=1591813800&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-06-11 00:00:00  ->  2020-06-12 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1591813800&before=1591900200&subreddit=bitcoin
count  0
1

count  0
90
2020-06-25 23:10:37
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1593106837&before=1593109800&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-06-26 00:00:00  ->  2020-06-27 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1593109800&before=1593196200&subreddit=bitcoin
count  0
73
2020-06-26 23:44:36
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1593195276&before=1593196200&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-06-27 00:00:00  ->  2020-06-28 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1593196200&before=1593282600&subreddit=bitcoin
count  0
5

count  0
46
2020-07-12 23:08:52
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1594575532&before=1594578600&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-07-13 00:00:00  ->  2020-07-14 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1594578600&before=1594665000&subreddit=bitcoin
count  0
67
2020-07-13 23:37:26
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1594663646&before=1594665000&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-07-14 00:00:00  ->  2020-07-15 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1594665000&before=1594751400&subreddit=bitcoin
count  0
9

count  1
41
2020-07-28 23:48:39
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1595960319&before=1595961000&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-07-29 00:00:00  ->  2020-07-30 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1595961000&before=1596047400&subreddit=bitcoin
count  0
86
2020-07-29 23:48:50
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1596046730&before=1596047400&subreddit=bitcoin
--------------------------------------------------------------------------------
2020-07-30 00:00:00  ->  2020-07-31 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1596047400&before=1596133800&subreddit=bitcoin
count  0
8

## Sentiment Analysis

In [9]:
import pandas as pd
import flair
from textblob import TextBlob
import os
import datetime
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [10]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'
sid = SentimentIntensityAnalyzer()

nltk.download('vader_lexicon')

2020-12-03 18:55:08,287 loading file C:\Users\dheer\.flair\models\sentiment-en-mix-distillbert_3.1.pt


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
def get_sentiment_val_for_flair(sentiments):
    """
    parse input of the format [NEGATIVE (0.9284018874168396)] and return +ve or -ve float value
    :param sentiments:
    :return:
    """
    total_sentiment = str(sentiments)
    neg = 'NEGATIVE' in total_sentiment
    if neg:
        total_sentiment = total_sentiment.replace('NEGATIVE', '')
    else:
        total_sentiment = total_sentiment.replace('POSITIVE', '')

    total_sentiment = total_sentiment.replace('(', '').replace('[', '').replace(')', '').replace(']', '')

    val = float(total_sentiment)
    if neg:
        return -val
    return val

In [12]:
def get_sentiment_report(input_filename, output_filename):
    df = pd.read_csv(input_filename)
    df = df[['title', 'selftext', 'publish_date']]
    df = df.fillna('')

    df['text'] = df['title'] + ' ' + df['selftext']
    df.set_index('publish_date', inplace=True)
    df.drop(['title', 'selftext'], axis=1, inplace=True)

    for row_i, row in df.iterrows():
        tb_sentiment_polarity_dict = dict()
        tb_sentiment_subjectivity_dict = dict()
        flair_sentiment_dict = dict()

        sid_pos_dict = dict()
        sid_neg_dict = dict()
        sid_neu_dict = dict()
        sid_com_dict = dict()

        data = row['text']
        print(row_i)
        print(data[0:15])
        flair_s = flair.data.Sentence(data)
        flair_sentiment.predict(flair_s)
        flair_total_sentiment = flair_s.labels
        flair_val = get_sentiment_val_for_flair(flair_total_sentiment)

        flair_sentiment_dict[str(row_i)] = flair_val
        tb_sentiment_polarity_dict[str(row_i)] = TextBlob(data).sentiment[0]
        tb_sentiment_subjectivity_dict[str(row_i)] = TextBlob(data).sentiment[1]

        ss = sid.polarity_scores(data)
        sid_pos_dict[str(row_i)] = ss['pos']
        sid_neg_dict[str(row_i)] = ss['neg']
        sid_neu_dict[str(row_i)] = ss['neu']
        sid_com_dict[str(row_i)] = ss['compound']

        flair_df = pd.DataFrame.from_dict(flair_sentiment_dict, orient='index', columns=['reddit_flair'])
        flair_df.index.name = 'timestamp'

        tb_polarity_df = pd.DataFrame.from_dict(tb_sentiment_polarity_dict, orient='index',
                                                columns=['reddit_tb_polarity'])
        tb_polarity_df.index.name = 'timestamp'

        tb_subjectivity_df = pd.DataFrame.from_dict(tb_sentiment_subjectivity_dict, orient='index',
                                                    columns=['reddit_tb_subjectivity'])
        tb_subjectivity_df.index.name = 'timestamp'

        sid_pos_df = pd.DataFrame.from_dict(sid_pos_dict, orient='index',
                                            columns=['reddit_sid_pos'])
        sid_pos_df.index.name = 'timestamp'

        sid_neg_df = pd.DataFrame.from_dict(sid_neg_dict, orient='index',
                                            columns=['reddit_sid_neg'])
        sid_neg_df.index.name = 'timestamp'

        sid_neu_df = pd.DataFrame.from_dict(sid_neu_dict, orient='index',
                                            columns=['reddit_sid_neu'])
        sid_neu_df.index.name = 'timestamp'

        sid_com_df = pd.DataFrame.from_dict(sid_com_dict, orient='index',
                                            columns=['reddit_sid_com'])
        sid_com_df.index.name = 'timestamp'

        final_senti_df = pd.concat([flair_df, tb_polarity_df, tb_subjectivity_df, sid_pos_df, sid_neg_df, sid_neu_df,
        							sid_com_df], axis=1)

        if os.path.exists(output_filename):
            keep_header = False
        else:
            keep_header = True

        final_senti_df.to_csv(output_filename, mode='a', header=keep_header)

    return

In [13]:
def clean_sentiment_report(input_filename, output_filename):
    # drop duplicates and sort
    master_df = pd.read_csv(input_filename, index_col=0)
    master_df.index = pd.to_datetime(master_df.index)
    idx = np.unique(master_df.index, return_index=True)[1]
    master_df = master_df.iloc[idx]
    master_df.to_csv(output_filename)

In [14]:
def bucketize_sentiment_report(input_filename, output_filename):
    start_date_time_obj = datetime.datetime(2019, 11, 21, 0)
    end_date_time_obj = datetime.datetime(2020, 7,30, 0)
    hr1 = datetime.timedelta(hours=1)
    curr_date_time_obj = start_date_time_obj
    in_df = pd.read_csv(input_filename)


    out_dict = dict()

    while curr_date_time_obj <= end_date_time_obj:
        curr_timestamp = curr_date_time_obj.strftime(format=fmt)
        # print(curr_timestamp)
        # create data dict with all possible timestamps and dummy value of reddit_flair
        # reddit_flair is chosen just randomly as a placeholder
        out_dict[curr_timestamp] = 0
        curr_date_time_obj += hr1

    out_df = pd.DataFrame.from_dict(out_dict, orient='index',
                                    columns=['reddit_flair'])

    # print(out_dict)
    out_df.index.name = 'timestamp'
    # populate more colums
    out_df['reddit_flair_count'] = 0
    out_df['reddit_tb_polarity'] = 0
    out_df['reddit_tb_polarity_count'] = 0
    out_df['reddit_tb_subjectivity'] = 0
    out_df['reddit_tb_subjectivity_count'] = 0
    out_df['reddit_sid_neg'] = 0
    out_df['reddit_sid_neu'] = 0
    out_df['reddit_sid_com'] = 0
    out_df['reddit_sid_count'] = 0

    for i in range(len(in_df)):
        timestamp = in_df.loc[i, 'timestamp']
        out_key = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
        # timestamp is current plus few minutes or seconds, so collect all these data in the bucket of next hour
        out_key += hr1
        out_key = out_key.strftime(format='%Y-%m-%d %H:00:00')
        #print(out_key)
        # add up all values and count how many values we have added. In next pass we would normalize the values
        try:
            out_df.loc[out_key, 'reddit_flair'] += in_df.loc[i, 'reddit_flair']
            out_df.loc[out_key, 'reddit_flair_count'] += 1
            out_df.loc[out_key, 'reddit_tb_polarity'] += in_df.loc[i, 'reddit_tb_polarity']
            out_df.loc[out_key, 'reddit_tb_polarity_count'] += 1
            out_df.loc[out_key, 'reddit_tb_subjectivity'] += in_df.loc[i, 'reddit_tb_subjectivity']
            out_df.loc[out_key, 'reddit_tb_subjectivity_count'] += 1
            out_df.loc[out_key, 'reddit_sid_pos'] += in_df.loc[i, 'reddit_sid_pos']
            out_df.loc[out_key, 'reddit_sid_neg'] += in_df.loc[i, 'reddit_sid_neg']
            out_df.loc[out_key, 'reddit_sid_neu'] += in_df.loc[i, 'reddit_sid_neu']
            out_df.loc[out_key, 'reddit_sid_com'] += in_df.loc[i, 'reddit_sid_com']
            out_df.loc[out_key, 'reddit_sid_count'] += 1
        except:
            pass

    # make timestamp as a column and reindex the dataframe to make loc method happy
    out_df['timestamp'] = out_df.index
    out_df.index = range(len(out_df))

    for i in range(len(out_df)):
        #print(out_df.loc[i, 'timestamp'])
        # normalize the values
        if out_df.loc[i, 'reddit_flair_count'] == 0:
            out_df.loc[i, 'reddit_flair'] = 0
        else:
            out_df.loc[i, 'reddit_flair'] /= out_df.loc[i, 'reddit_flair_count']

        if out_df.loc[i, 'reddit_tb_polarity_count'] == 0:
            out_df.loc[i, 'reddit_tb_polarity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_polarity'] /= out_df.loc[i, 'reddit_tb_polarity_count']

        if out_df.loc[i, 'reddit_tb_subjectivity_count'] == 0:
            out_df.loc[i, 'reddit_tb_subjectivity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_subjectivity'] /= out_df.loc[i, 'reddit_tb_subjectivity_count']

        if out_df.loc[i, 'reddit_sid_count'] == 0:
            out_df.loc[i, 'reddit_sid_pos'] = 0
            out_df.loc[i, 'reddit_sid_neg'] = 0
            out_df.loc[i, 'reddit_sid_neu'] = 0
            out_df.loc[i, 'reddit_sid_com'] = 0
        else:
            out_df.loc[i, 'reddit_sid_pos'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neg'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neu'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_com'] /= out_df.loc[i, 'reddit_sid_count']

        if os.path.exists(output_filename):
            keep_header = False
        else:
            keep_header = True

    out_df.drop(['reddit_flair_count', 'reddit_tb_polarity_count', 'reddit_tb_subjectivity_count','reddit_sid_count'], axis=1,
                inplace=True)
    # change back index to timestamp to save the data in csv
    out_df.set_index('timestamp', inplace=True)
    out_df.to_csv(output_filename)


In [16]:
input_filename = 'reddit_data.csv'
output_sentiment_filename = input_filename[0:-4] + '_sentiment.csv'

# read input_filename (which can be generated by download_data_from_reddit.py script) and performs
# sentiment analyis of the text data
# get_sentiment_report(input_filename, output_sentiment_filename)
output_sentiment_bucketize_filename = output_sentiment_filename[0:-4] + '_bucketized.csv'

# reddit posts can land anytime. Collect all the posts (and its sentiment reports) landed on a given hour (0 to 59 minutes)
# and bucketize them all into the corresponding hour
bucketize_sentiment_report(output_sentiment_filename, output_sentiment_bucketize_filename)

2019-11-21 00:26:49
Hi I’m new to t
2019-11-21 01:27:42
Bitcoin pls go 
2019-11-21 01:48:19
The CEO of Payp
2019-11-21 02:22:03
Need Help: How 
2019-11-21 02:44:39
Bloomberg Panel
2019-11-21 02:44:41
PayPal CEO Hodl
2019-11-21 02:48:37
Shill me your f
2019-11-21 02:50:15
do u believe th
2019-11-21 02:51:21
I got this in m
2019-11-21 02:56:06
It's getting fr
2019-11-21 03:18:25
Selling $100 wo
2019-11-21 03:32:18
Bitcoin Mining 
2019-11-21 04:33:34
Banks are SCREW
2019-11-21 04:33:54
EXPLAINED - Bit
2019-11-21 05:42:40
Electrum Bitcoi
2019-11-21 06:15:54
WORLD WIDE WEST
2019-11-21 06:49:01
Bitcoin Officia
2019-11-21 06:59:43
Number of Bitco
2019-11-21 07:30:06
Roger Ver Annou
2019-11-21 08:09:35
Bitcoin Price D
2019-11-21 08:21:09
Bloomberg Panel
2019-11-21 08:25:56
Binance acquire
2019-11-21 08:28:13
The Cryptoconom
2019-11-21 08:29:41
Many People sti
2019-11-21 08:46:50
Top Bitcoin Pri
2019-11-21 09:01:23
Kraken OTC’s 8 
2019-11-21 09:36:58
The 4 Hardware 
2019-11-21 10:09:30
1 Gigawa

2019-11-23 14:58:17
Bitcoin Fan Art
2019-11-23 15:26:19
China Has to Pe
2019-11-23 16:19:36
Manage Volatili
2019-11-23 16:40:53
Good morning bi
2019-11-23 16:44:31
Bitcoin Price: 
2019-11-23 16:44:31
Bitcoin Price M
2019-11-23 17:12:13
Bitcoin Could B
2019-11-23 17:16:35
Bitcoin Network
2019-11-23 17:29:47
Has Bitcoin fai
2019-11-23 17:41:40
Gleb Naumenko -
2019-11-23 17:53:56
ESPN took my Bi
2019-11-23 18:13:53
ESPN took my Pr
2019-11-23 18:29:39
Announcements f
2019-11-23 18:44:46
Bitcoin Price D
2019-11-23 18:52:15
Are all possibl
2019-11-23 19:01:17
bitcoin price i
2019-11-23 19:26:23
The Chaos Theor
2019-11-23 19:38:59
Can I buy bitco
2019-11-23 19:48:19
For those who d
2019-11-23 20:13:30
How to respond 
2019-11-23 20:21:02
President Trump
2019-11-23 21:04:06
Why does forbes
2019-11-23 22:10:13
Without Bitcoin
2019-11-23 22:19:22
Without Bitcoin
2019-11-23 22:38:06
Why The World N
2019-11-23 22:48:34
Receiving Bitco
2019-11-23 22:54:05
Who is the rich
2019-11-23 23:11:34
I really

2019-11-26 16:06:23
In a recent Mr 
2019-11-26 16:16:48
In the lastest 
2019-11-26 16:18:01
Despite Crash, 
2019-11-26 16:36:04
This Bitcoin Mo
2019-11-26 16:42:15
“Plan B” the tw
2019-11-26 16:49:12
Bitcoin Black F
2019-11-26 16:59:09
Technology alwa
2019-11-26 17:12:00
If you know you
2019-11-26 17:16:42
Bitcoin Network
2019-11-26 17:31:09
YouTube Demonet
2019-11-26 18:12:38
Bitcoin-Based I
2019-11-26 18:23:50
Great use case 
2019-11-26 18:25:10
Bitcoin ATM Fir
2019-11-26 18:31:33
A beginner's gu
2019-11-26 18:38:35
Bitcoin Versus 
2019-11-26 18:58:46
Boom it happene
2019-11-26 19:05:31
Bitcoin scam on
2019-11-26 19:07:24
Any body else n
2019-11-26 19:29:32
'Bitcoin' Suppo
2019-11-26 19:31:36
Bitcoin ATM Fir
2019-11-26 19:33:45
Get your untrac
2019-11-26 19:37:23
Recover Cold Bi
2019-11-26 20:00:30
Bitcoin reaches
2019-11-26 20:15:16
BTCUSD - 26/11/
2019-11-26 20:31:18
Double Your Bit
2019-11-26 20:41:19
Double Your Bit
2019-11-26 21:01:37
How to Earn and
2019-11-26 21:03:19
One thin

2019-11-29 10:33:12
61% of Bitcoin 
2019-11-29 10:54:39
-&gt; Bitcoin F
2019-11-29 11:00:27
Bitcoin Adverti
2019-11-29 11:02:25
New Law Propose
2019-11-29 12:07:08
Breaking: Germa
2019-11-29 12:09:18
GET YOUR FREE B
2019-11-29 12:12:41
If anyone is lo
2019-11-29 12:30:29
Note to Bisq us
2019-11-29 12:36:04
A very low leve
2019-11-29 12:38:33
Buy Bitcoin wit
2019-11-29 12:39:16
Germany: New Pr
2019-11-29 12:39:18
Bitcoin Price G
2019-11-29 12:48:54
Currently, as b
2019-11-29 12:49:46
Buy bitcoin usi
2019-11-29 12:51:44
Ethereum can de
2019-11-29 12:51:46
Let Me Tell You
2019-11-29 12:55:57
Millions are pr
2019-11-29 13:34:23
Grayscale is ma
2019-11-29 13:37:06
The Bitcoin Cal
2019-11-29 13:55:49
The price of Bi
2019-11-29 14:02:07
Miners Keep Loy
2019-11-29 14:02:39
You Can Not Giv
2019-11-29 14:10:53
A reminder why 
2019-11-29 14:13:10
Additional fait
2019-11-29 14:21:47
What is the bes
2019-11-29 14:22:13
Do they use Bit
2019-11-29 14:24:49
Why Bitcoin? 
2019-11-29 14:28:24
Germany: N

2019-12-02 15:39:56
3 Ways Congress
2019-12-02 15:45:12
Bitcoin Invoice
2019-12-02 15:48:10
Bitcoin News, A
2019-12-02 15:49:13
Does Bitcoin Ne
2019-12-02 15:51:06
After 11 years,
2019-12-02 15:54:39
The majority of
2019-12-02 16:02:05
10 Years of Con
2019-12-02 16:04:13
Bitcoin P2P Mar
2019-12-02 16:15:47
64 Percent of B
2019-12-02 16:26:28
1934 Silver Cer
2019-12-02 16:34:09
How to identify
2019-12-02 16:34:56
(BITCOIN SKETCH
2019-12-02 17:10:57
Gift Off, one o
2019-12-02 17:14:15
Bitcoin HODLers
2019-12-02 17:16:56
Bitcoin Network
2019-12-02 17:34:54
Understanding B
2019-12-02 17:46:31
Most people don
2019-12-02 17:48:24
Como Sacar Bitc
2019-12-02 17:48:38
Do Bitcoin Comp
2019-12-02 17:52:25
What's the best
2019-12-02 18:10:09
Have two people
2019-12-02 18:48:45
The last Bitcoi
2019-12-02 18:56:08
Will Bitcoin Co
2019-12-02 19:07:54
Financial Uncer
2019-12-02 19:12:18
Will Bitcoin co
2019-12-02 19:18:25
When I see the 
2019-12-02 19:19:38
The Best Bitcoi
2019-12-02 19:34:23
Hong Kon

...

2020-07-18 21:10:26
Check out my ne
2020-07-18 21:25:39
"Despite the ne
2020-07-18 21:37:45
Additional apps
2020-07-18 22:37:06
Venezuela Bans 
2020-07-18 23:00:13
Bitcoin and the
2020-07-18 23:22:18
Passive Income 
2020-07-18 23:46:03
How to understa
2020-07-19 00:11:36
There's a very 
2020-07-19 00:49:55
What happens wh
2020-07-19 01:18:36
Banks are compl
2020-07-19 01:29:11
Keiser Report |
2020-07-19 02:18:34
Twitter Hack sk
2020-07-19 02:45:25
How to buy bitc
2020-07-19 03:10:56
Privacy on Bitc
2020-07-19 03:16:04
Bitcoin logo co
2020-07-19 03:21:50
All Bitcoin Lov
2020-07-19 03:50:58
Running my own 
2020-07-19 03:58:38
bitcoin is bori
2020-07-19 04:14:43
Newbie to bitco
2020-07-19 04:22:58
With the first 
2020-07-19 04:50:22
Bitcoin gamblin
2020-07-19 05:57:59
[Forbes] A Top 
2020-07-19 06:37:07
$6M Worth of Te
2020-07-19 07:54:18
B for Bitcoin 
2020-07-19 08:01:29
Bitcoin lottery
2020-07-19 08:07:34
Bitcoin Extreme
2020-07-19 08:17:31
Bitcoin (right 
2020-07-19 09:23:04
Which Bit

2020-07-22 07:43:41
Free Bitcoin mi
2020-07-22 08:37:29
Third Anniversa
2020-07-22 08:43:04
CZ Binance CEO 
2020-07-22 09:43:37
Crypto Bitcoin 
2020-07-22 10:04:59
The Bitcoin Ral
2020-07-22 10:13:45
How can I find 
2020-07-22 10:30:47
Blockchain Land
2020-07-22 10:44:55
Comments from R
2020-07-22 11:37:27
Here’s How to C
2020-07-22 12:34:27
Bitcoin protect
2020-07-22 13:30:48
Bitcoin develop
2020-07-22 13:35:41
100x Group is d
2020-07-22 14:40:55
I think Satoshi
2020-07-22 15:34:43
Bitcoin USD dif
2020-07-22 16:04:49
Here's How to C
2020-07-22 16:06:16
Others are NON-
2020-07-22 16:15:13
Bitcoin app Is 
2020-07-22 16:49:45
Kashif Raza: Tw
2020-07-22 17:04:29
"With the disco
2020-07-22 18:04:17
"To me Bitcoin 
2020-07-22 18:07:46
MasterCard to s
2020-07-22 18:56:06
We decided that
2020-07-22 19:01:02
P2P Bitcoin vol
2020-07-22 19:04:36
We decided that
2020-07-22 19:06:15
Coinsquare Exch
2020-07-22 19:10:06
We turned our i
2020-07-22 19:10:13
I want to hold 
2020-07-22 19:19:10
where to

2020-07-25 07:37:10
Bitcoin Suisse 
2020-07-25 08:26:42
Chess Ex-Champi
2020-07-25 09:28:24
/r/bitcoin cele
2020-07-25 09:29:23
For each upvote
2020-07-25 09:39:17
Bitcoin logo Do
2020-07-25 09:50:03
Use Bitcoin - A
2020-07-25 10:30:40
Bitcoin busines
2020-07-25 11:19:41
Is Bitcoin Read
2020-07-25 11:38:01
Earn Free Bitco
2020-07-25 12:15:03
why hasn't bitc
2020-07-25 12:28:30
Russian bankers
2020-07-25 12:52:19
How would you f
2020-07-25 13:05:25
Dollar averagin
2020-07-25 13:07:17
Russian bankers
2020-07-25 13:24:07
Indian Top YouT
2020-07-25 13:25:53
Indian Top YouT
2020-07-25 13:55:46
Finally made it
2020-07-25 14:02:51
India's Popular
2020-07-25 14:29:15
Bill Gates: I w
2020-07-25 15:02:53
Bitcoin Rapid-F
2020-07-25 15:24:58
Friends, let me
2020-07-25 15:54:30
Are there real 
2020-07-25 16:37:21
Us Federal Cour
2020-07-25 17:01:06
Send Bitcoin To
2020-07-25 17:10:23
Who has read Th
2020-07-25 17:10:30
Tech question. 
2020-07-25 17:17:40
Bitcoin Wallets
2020-07-25 18:53:34
Why The 

2020-07-28 03:53:11
The best music 
2020-07-28 03:57:49
Bitcoin will ab
2020-07-28 04:01:00
Guy: What shoul
2020-07-28 04:04:27
Its official, 1
2020-07-28 04:05:59
Should I start 
2020-07-28 04:06:12
Comprar bitcoin
2020-07-28 04:12:23
* Hot off the p
2020-07-28 04:14:50
Thought exercis
2020-07-28 04:35:15
Any prediction 
2020-07-28 04:42:10
1 BTC = 5.2 mil
2020-07-28 05:16:38
question for bi
2020-07-28 05:18:20
How much volume
2020-07-28 05:21:05
Bitcoin goes to
2020-07-28 05:44:28
Phoenix: Bitcoi
2020-07-28 05:47:25
Bitcoin increas
2020-07-28 06:03:29
Sending Bitcoin
2020-07-28 06:05:03
Bitcoin is goin
2020-07-28 06:07:16
What are the ta
2020-07-28 06:32:03
PSA dont get gr
2020-07-28 06:36:00
How can i mine 
2020-07-28 06:37:18
I try to send b
2020-07-28 06:37:28
Bitcoin Fees Sk
2020-07-28 06:37:55
BTC News Today 
2020-07-28 06:37:58
Bitcoin ATM spo
2020-07-28 06:46:06
Bitcoin help I’
2020-07-28 06:59:39
No one can expl
2020-07-28 07:04:21
Bitcoin IRAs 
2020-07-28 08:05:07
What's a g

2020-07-30 11:15:10
Compacting a 29
2020-07-30 11:23:13
😮 Bitpaint VR i
2020-07-30 12:09:47
Bit upset with 
2020-07-30 12:23:38
Bitcoin could g
2020-07-30 13:34:51
Bitcoin 11K 
2020-07-30 13:44:56
Bitcoin Price P
2020-07-30 14:06:05
Satoshis 1.1 mi
2020-07-30 14:15:14
Earn bitcoin by
2020-07-30 14:42:56
ASK REDDIT (Tec
2020-07-30 14:44:27
Introducing Min
2020-07-30 14:46:37
Which wallet is
2020-07-30 14:49:33
Mining BITCOIN 
2020-07-30 14:53:01
The Bitcoin Lay
2020-07-30 15:36:47
Bitcoin Mining 
2020-07-30 16:23:32
Analysts Predic
2020-07-30 16:34:47
Fun with Bitcoi
2020-07-30 16:48:25
How do I store 
2020-07-30 16:49:00
Fidelity Bitcoi
2020-07-30 16:59:31
Fidelity gives 
2020-07-30 17:12:02
ANNOUNCING MINS
2020-07-30 18:04:35
Andreas Antonop
2020-07-30 18:36:52
Every time bitc
2020-07-30 18:37:15
‘Huge Demand in
2020-07-30 18:41:57
How are your op
2020-07-30 19:03:56
Why You Should 
2020-07-30 19:11:13
An open letter 
2020-07-30 20:38:48
Black Bitcoin 
2020-07-30 20:50:16
Authorities 